In [ ]:
import pandas as pd
import numpy as np
from keras.preprocessing.text import Tokenizer
from keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.utils import to_categorical
from keras.models import Sequential
from keras.layers import LSTM, Embedding, Dense
from tensorflow.keras.callbacks import ModelCheckpoint
from tensorflow.keras.utils import to_categorical 
from sklearn.model_selection import train_test_split
from keras.models import load_model
import tensorflow as tf
import re
from nltk.tokenize import word_tokenize
from nltk.stem import WordNetLemmatizer
from nltk.corpus import stopwords
from sklearn.model_selection import train_test_split
from tensorflow.keras.models import Sequential, load_model
from tensorflow.keras.layers import Embedding, LSTM, Dense, Dropout
from keras.callbacks import EarlyStopping
from keras import regularizers
import ast

In [ ]:
# Remove numbers from text
def remove_numbers(text):
    return re.sub(r'\d+', '', text)
def remove_unusual_characters(text):
    res = re.sub(r'[^\w\s,.;:!?\-]','',text)
    res = re.sub(r'\.+',' ',res)
    res = re.sub(r'!+','',res)
    res = re.sub(r',+',' ',res)
    res = re.sub(r';+',' ',res)
    res = re.sub(r';+',' ',res)
    res = re.sub(r'\n+','',res)
    res = re.sub(r'\d+','',res)
    return re.sub(r'\s+', ' ', res)


    
    return res
def convert_to_lowercase(text):
    return text.lower()

In [ ]:
df=pd.read_csv("srl_sentiment_labeled_data.csv")

In [ ]:
df

In [ ]:
df = df[['Review','Label','sentiment','srl label']]


In [ ]:
df = df.dropna(subset='Review')


In [ ]:
df['Review'] = df['Review'].apply(remove_numbers)
df['Review']=df['Review'].apply(remove_unusual_characters)
df['Review']=df['Review'].apply(convert_to_lowercase)

In [ ]:
tokenizer = Tokenizer()
tokenizer.fit_on_texts(df['Review'])
sequences = tokenizer.texts_to_sequences(df['Review'])
max_length = max(len(seq) for seq in sequences)

In [ ]:
max_length

In [ ]:
tokenizer = tf.keras.preprocessing.text.Tokenizer()

# Fit tokenizer on text
tokenizer.fit_on_texts(df['Review'])

# Convert text to sequences
sequences = tokenizer.texts_to_sequences(df['Review'])
word_index = tokenizer.word_index
print(f'Found {len(word_index)} unique tokens.')
# Print sequences
print("Tokenized sequences:")
 
padded_sequences = tf.keras.preprocessing.sequence.pad_sequences(sequences, padding='post')
type(padded_sequences)

In [ ]:
srllabels=df['Label'].to_dict()
for i in range(len(srllabels)):
    srllabels[i] = ast.literal_eval(srllabels[i])

srl = []
# Assign indices for each label
for i in range(len(srllabels)):
    array = [0] * 33
    for label, indices in srllabels[i].items():
        if label == 'PRED':
            for idx in indices:
                array[idx] = 1
        elif label == 'PROD1' or label == 'PROD2':
            for idx in indices:
                array[idx] = 2
        elif label == 'ASP':
            for idx in indices:
                array[idx] = 3
        #elif label == 'PROD2':
        #    for idx in indices:
        #        array[idx] = 4
    srl.append(array)
srl = np.array(srl)
df['srl'] = [l for l in srl]

In [ ]:
#not sure if we need this
#results=[]
#for seq,re in zip(padded_sequences,res):
#    new_re = np.append(re, [0, 0])
#    results.append(new_re)

# print(results
for seqi,rei in zip(padded_sequences,df['srl']): 
    print(len(seqi),len(rei))

In [ ]:
input_seq_model = np.zeros((len(padded_sequences), len(padded_sequences[0]), 2))
for i in range(len(padded_sequences)):
    for j in range(len(padded_sequences[0])):
        # print(i,j)
        input_seq_model[i][j] = [padded_sequences[i][j], df['srl'][i][j]]
        

print(input_seq_model.shape)
input_seq_model[0]
        
    
    

In [ ]:
# Label and encode ratings
df_encoded = pd.get_dummies(df['sentiment'])

In [ ]:
# Define the X and y values
X = input_seq_model
y = df_encoded.values

In [ ]:
X_train, X_test, y_train, y_test = train_test_split(X, y, train_size=0.7)

In [ ]:
from tensorflow.keras.callbacks import ModelCheckpoint
from tensorflow import convert_to_tensor
# Define validation set and convert to Tensor
stop = int(len(X_test)/2)

X_train = convert_to_tensor(X_train)
X_val = convert_to_tensor(X_test[:stop])
X_test = convert_to_tensor(X_test[stop:])

y_train = convert_to_tensor(y_train)
y_val = convert_to_tensor(y_test[:stop])
y_test = convert_to_tensor(y_test[stop:])
checkpoint_path = "model_checkpoint.h5"
checkpoint_callback = ModelCheckpoint(filepath=checkpoint_path,
                                      monitor='val_accuracy',
                                      save_best_only=True,
                                      verbose=1)

In [ ]:
import numpy as np
from keras.models import Sequential
from keras.layers import LSTM, Dense, Concatenate, Input
from keras.models import Model

# Assuming you have your data stored in variables padded_sequences and encoded_results
padded_sequences_shape = (33,2)

input1 = Input(shape = input_seq_model.shape[1:])
# embedding_layer = Embedding(input_dim=6330,  # Replace vocabulary_size with your actual vocabulary size
#                             output_dim=200,  # Replace embedding_dim with your desired embedding dimension
#                             input_length=36)(input1)
# Create the rest of the model
x = LSTM(128,return_sequences = True)(input1)
x = Dropout(0.5)(x)  # Add a dropout layer with a dropout rate of 0.5
x = LSTM(128)(x)
x = Dense(64,  activation='relu', 
          kernel_regularizer=regularizers.l2(0.01), 
          activity_regularizer=regularizers.l1(0.01))(x)
output = Dense(2, activation='softmax')(x)

# Create the model
model = Model(inputs=[input1], outputs=output)

# Compile the model
model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])

# Print model summary
model.summary()


In [ ]:
#import save model


In [ ]:
history = model.fit(
    X_train, y_train,
    epochs=50,
    batch_size=32,
    validation_data=(X_val, y_val),
    callbacks=[checkpoint_callback]
)


In [ ]:
from sklearn.metrics import classification_report

# Assuming `model` is already trained and `X_test`, `y_test` are the test data
# Make predictions
model = load_model('SRL_Sentiment_model.h5')
y_pred = model.predict(X_val)

# Convert predictions to classes
y_pred_classes = np.argmax(y_pred, axis=1)

# If your labels are one-hot encoded, you may want to convert them to integer labels
y_true_classes = np.argmax(y_val, axis=1)

# Generate classification report
report = classification_report(y_true_classes, y_pred_classes)

# Print the report
print(report)
